In [1]:
import numpy as np
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_1D import *
from ids import *
%matplotlib inline

In [2]:
### how to make from permatation matrix readable array
def perm_array(A):
    p_a = np.array((A.shape[0]))
    p_a = np.argmax(A, axis = 1)
    return p_a

In [3]:
n = 4 #nmber of sampling points   
deg = 3 # Degree of polynomiapaired plu Last Checkpoint: 22 minutes ago (unsaved changes)l          
interval = [-3, 3]  # interval of x
x = np.linspace(interval[0], interval[1], n, dtype=np.float64)
p_size = deg+1

In [4]:
### generating matrix with special structure
b = Gen_1D_coupled(p_size, x, debug=True)
b

number of points(l) = 4,  number of monoms(n_size) = 4


array([[   1.,   -3.,   17.,  -99.],
       [   0.,    1.,  -12.,  105.],
       [   1.,   -1.,    1.,   -1.],
       [   0.,    1.,   -4.,    9.],
       [   1.,    1.,    1.,    1.],
       [   0.,    1.,    4.,    9.],
       [   1.,    3.,   17.,   99.],
       [   0.,    1.,   12.,  105.]])

In [5]:
p,l,u,q = pluq_ids(b)

[[   1.    0.]
 [  -3.    1.]
 [  17.  -12.]
 [ -99.  105.]]
[[ 1.  0.]
 [-1.  1.]
 [ 1. -4.]
 [-1.  9.]]
[[ 1.  0.]
 [ 1.  1.]
 [ 1.  4.]
 [ 1.  9.]]
[[   1.    0.]
 [   3.    1.]
 [  17.   12.]
 [  99.  105.]]
correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 0, 1)
('column coordinates:', 2, 3)
('maxvol 2x2 submatrix', array([[  17.,  -12.],
       [ -99.,  105.]]))
('with det = ', 596.99999999999955)
pivoting and permutations start...
[[  17.   -3.    1.  -99.]
 [ -12.    1.    0.  105.]
 [   1.   -1.    1.   -1.]
 [  -4.    1.    0.    9.]
 [   1.    1.    1.    1.]
 [   4.    1.    0.    9.]
 [  17.    3.    1.   99.]
 [  12.    1.    0.  105.]]
[[  17.           -3.            1.          -99.        ]
 [   0.           -1.11764706    0.70588235   35.11764706]
 [   0.           -0.82352941    0.94117647    4.82352941]
 [   0.            0.29411765    0.23529412  -14.29411765]
 [   0.            1.17647059    0.94117647    6.82352941]
 [   0.            1.70588235   

In [6]:
### here we can see that in the first 4 rows blocked structure was saved, i.e. func-deriv, func-deriv
perm_array(p)

array([0, 1, 6, 7, 4, 5, 2, 3])

In [ ]:
### maxvol preprocessing
ids = np.dot(p,np.dot(b,q))[:p_size]
A = np.dot(p,np.dot(b,q))

temp = np.dot(A,np.linalg.inv(ids))

In [ ]:
### here we are checking all pairs under Identity matrix and try to find 2x2 block with det>1+tol
tol = 0.05
n = temp.shape[0]
m = temp.shape[1]
for k in range(m,n,2):
        pair = np.concatenate((temp[k,:],temp[k+1,:])).reshape(2,m).T
        piv,_ = maxvol(pair)
        piv = np.sort(piv)
        print pair, pair[piv]
        if (np.abs(np.linalg.det(pair[piv])) > (1 + tol)):
            print('exist!')

# RGV

In [4]:
mat_test = np.array([ 
    [50,  0 , 0,0,0, 1],
    [0, 0, 30,  0 ,0, 3],
    [0,0, 0, 20,  0 , 11],
    [0, 40,  0 ,0,0, 2],
    [0,0, 0,0,  10,  12],
    [0,0,0, 0,0, 50]
], dtype=float)
print (mat_test)
print ()

[[ 50.   0.   0.   0.   0.   1.]
 [  0.   0.  30.   0.   0.   3.]
 [  0.   0.   0.  20.   0.  11.]
 [  0.  40.   0.   0.   0.   2.]
 [  0.   0.   0.   0.  10.  12.]
 [  0.   0.   0.   0.   0.  50.]]
()


In [5]:
p,l,u,q = pluq_ids(mat_test, debug=True)

[[ 50.   0.]
 [  0.   0.]
 [  0.  30.]
 [  0.   0.]
 [  0.   0.]
 [  1.   3.]]
[[  0.   0.]
 [  0.  40.]
 [  0.   0.]
 [ 20.   0.]
 [  0.   0.]
 [ 11.   2.]]
[[  0.   0.]
 [  0.   0.]
 [  0.   0.]
 [  0.   0.]
 [ 10.   0.]
 [ 12.  50.]]
correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 0, 1)
('column coordinates:', 0, 2)
('maxvol 2x2 submatrix', array([[ 50.,   0.],
       [  0.,  30.]]))
('with det = ', 1500.0)
pivoting and permutations start...
[[ 50.   0.   0.   0.   0.   1.]
 [  0.   0.  30.   0.   0.   3.]
 [  0.   0.   0.  20.   0.  11.]
 [  0.  40.   0.   0.   0.   2.]
 [  0.   0.   0.   0.  10.  12.]
 [  0.   0.   0.   0.   0.  50.]]
[[ 50.   0.   0.   0.   0.   1.]
 [  0.   0.  30.   0.   0.   3.]
 [  0.   0.   0.  20.   0.  11.]
 [  0.  40.   0.   0.   0.   2.]
 [  0.   0.   0.   0.  10.  12.]
 [  0.   0.   0.   0.   0.  50.]]
[[ 50.   0.   0.   0.   0.   1.]
 [  0.  30.   0.   0.   0.   3.]
 [  0.   0.   0.  20.   0.  11.]
 [  0.   0.  40.   0.   0.   2.]
 [  0

In [6]:
perm_array(p)

array([0, 1, 2, 3, 4, 5])

In [8]:
mat_test_1 = np.array([ 
    [0,0, 0, 20,  0 , 11],
    [0, 40,  0 ,0,0, 2],    
    [50,  0 , 0,0,0, 1],
    [0, 0, 30,  0 ,0, 3],
    [0,0, 0,0,  10,  12],
    [0,0,0, 0,0, 50]
], dtype=float)
print (mat_test_1)
print ()

[[  0.   0.   0.  20.   0.  11.]
 [  0.  40.   0.   0.   0.   2.]
 [ 50.   0.   0.   0.   0.   1.]
 [  0.   0.  30.   0.   0.   3.]
 [  0.   0.   0.   0.  10.  12.]
 [  0.   0.   0.   0.   0.  50.]]
()


In [9]:
p,l,u,q = pluq_ids(mat_test_1, debug=True)

[[  0.   0.]
 [  0.  40.]
 [  0.   0.]
 [ 20.   0.]
 [  0.   0.]
 [ 11.   2.]]
[[ 50.   0.]
 [  0.   0.]
 [  0.  30.]
 [  0.   0.]
 [  0.   0.]
 [  1.   3.]]
[[  0.   0.]
 [  0.   0.]
 [  0.   0.]
 [  0.   0.]
 [ 10.   0.]
 [ 12.  50.]]
correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 2, 3)
('column coordinates:', 0, 2)
('maxvol 2x2 submatrix', array([[ 50.,   0.],
       [  0.,  30.]]))
('with det = ', 1500.0)
pivoting and permutations start...
[[ 50.   0.   0.   0.   0.   1.]
 [  0.  40.   0.   0.   0.   2.]
 [  0.   0.   0.  20.   0.  11.]
 [  0.   0.  30.   0.   0.   3.]
 [  0.   0.   0.   0.  10.  12.]
 [  0.   0.   0.   0.   0.  50.]]
[[ 50.   0.   0.   0.   0.   1.]
 [  0.  40.   0.   0.   0.   2.]
 [  0.   0.   0.  20.   0.  11.]
 [  0.   0.  30.   0.   0.   3.]
 [  0.   0.   0.   0.  10.  12.]
 [  0.   0.   0.   0.   0.  50.]]
[[ 50.   0.   0.   0.   0.   1.]
 [  0.  30.   0.   0.   0.   3.]
 [  0.   0.   0.  20.   0.  11.]
 [  0.   0.  40.   0.   0.   2.]
 [  0

In [4]:
mat_test_2 = np.array([ 
    [0,0, 0, 20, 0 , 11],
    [0, 0,  0 ,0,0.043, 2],    
    [40,  0 , 0,0,23, 1],
    [0, 30, 0,  0 ,0, 0],
    [0,0, 20,0,  90,  12],
    [0,0,0, 0,20, 0]
], dtype=float)
print (mat_test_2)
print ()

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   2.00000000e+01
    0.00000000e+00   1.10000000e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    4.30000000e-02   2.00000000e+00]
 [  4.00000000e+01   0.00000000e+00   0.00000000e+00   0.00000000e+00
    2.30000000e+01   1.00000000e+00]
 [  0.00000000e+00   3.00000000e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   2.00000000e+01   0.00000000e+00
    9.00000000e+01   1.20000000e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    2.00000000e+01   0.00000000e+00]]
()


In [9]:
perm_array(p)

array([2, 3, 0, 1, 4, 5])

In [5]:
np.linalg.det(mat_test_2)

-19199999.999999933

In [6]:
p,l,u,q = pluq_ids(mat_test_2, debug=True)

[[  0.      0.   ]
 [  0.      0.   ]
 [  0.      0.   ]
 [ 20.      0.   ]
 [  0.      0.043]
 [ 11.      2.   ]]
[[ 40.   0.]
 [  0.  30.]
 [  0.   0.]
 [  0.   0.]
 [ 23.   0.]
 [  1.   0.]]
[[  0.   0.]
 [  0.   0.]
 [ 20.   0.]
 [  0.   0.]
 [ 90.  20.]
 [ 12.   0.]]
correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 2, 3)
('column coordinates:', 0, 1)
('maxvol 2x2 submatrix', array([[ 40.,   0.],
       [  0.,  30.]]))
('with det = ', 1200.0)
pivoting and permutations start...
[[  4.00000000e+01   0.00000000e+00   0.00000000e+00   0.00000000e+00
    2.30000000e+01   1.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    4.30000000e-02   2.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   2.00000000e+01
    0.00000000e+00   1.10000000e+01]
 [  0.00000000e+00   3.00000000e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   2.00000000e+01   0.00000000e+00
 

In [7]:
perm_array(p)

array([2, 3, 4, 5, 0, 1])

In [11]:
mat_test_3 = np.array([ 
    [20,0, 0, 2, 0 , 11],
    [0, 10,  0 ,0,0.043, 2],    
    [0,  0 , 30,0,15, 1],
    [0, 0, 0,  10 ,0, 0],
    [0,0, 20,0,  90,  12],
    [0,0,0, 0,0, 1900]
], dtype=float)
print (mat_test_3.astype(int))
print ()

[[  20    0    0    2    0   11]
 [   0   10    0    0    0    2]
 [   0    0   30    0   15    1]
 [   0    0    0   10    0    0]
 [   0    0   20    0   90   12]
 [   0    0    0    0    0 1900]]
()


In [12]:
p,l,u,q = pluq_ids(mat_test_3, debug=True)

[[ 20.      0.   ]
 [  0.     10.   ]
 [  0.      0.   ]
 [  2.      0.   ]
 [  0.      0.043]
 [ 11.      2.   ]]
[[  0.   0.]
 [  0.   0.]
 [ 30.   0.]
 [  0.  10.]
 [ 15.   0.]
 [  1.   0.]]
[[    0.     0.]
 [    0.     0.]
 [   20.     0.]
 [    0.     0.]
 [   90.     0.]
 [   12.  1900.]]
correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 4, 5)
('column coordinates:', 4, 5)
('maxvol 2x2 submatrix', array([[   90.,     0.],
       [   12.,  1900.]]))
('with det = ', 171000.00000000003)
pivoting and permutations start...
[[  9.00000000e+01   0.00000000e+00   2.00000000e+01   0.00000000e+00
    0.00000000e+00   1.20000000e+01]
 [  4.30000000e-02   1.00000000e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   2.00000000e+00]
 [  1.50000000e+01   0.00000000e+00   3.00000000e+01   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+01
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.0000

In [13]:
perm_array(p)

array([4, 5, 2, 3, 0, 1])

# RGV 2

In [2]:
mat_test = np.array([
    [ 1000, 0, 0, 0,0,0 ],
    [0,3,0,0,0,1],
    [0, 0 , 1000, 0, 0, 0] , 
    [0, 0, 0, 2, 0, 0],
    [0, 0, 0, 0, 1000, 0] , 
    [0,3,0,2,0,1],
    [1,2,3,4,0,0],
    [4,3,2,1,0,1]
], dtype=float)

## Check, that matrix is good

In [3]:
np.linalg.matrix_rank(mat_test)

6

In [4]:
np.linalg.det(mat_test[[0,1,4,5,6,7]])

-7999999.9999999702

In [6]:
p,l,u,q, info = pluq_ids(mat_test, debug=False)
print info

[0 1]


# Another Error 

In [36]:
mat_test_5 = np.array([[18,  7, 10, 14, 19, 12],
       [16, 15,  5,  6, 17, 15],
       [15,  0,  7,  7, 12, 11],
       [12,  4,  6, 13,  6,  5],
       [19,  8, 10, 16,  3, 16],
       [10,  3,  5,  1,  1,  4],
       [ 0,  7,  7,  6, 17,  9],
       [14,  7,  9, 16, 17,  5],
       [18, 18, 19,  3,  9, 12],
       [12,  7, 16,  1,  0,  2]], dtype=float)

In [37]:
p,l,u,q = pluq_ids(mat_test_5, debug=True)

('on the', 0, 'slice')
('best row block is', 6, 7)
('column coordinates:', 0, 4)
('maxvol 2x2 submatrix', array([[  0.,  14.],
       [ 17.,  17.]]))
('with det = ', 237.99999999999986)
pivoting and permutations start...
[[ 17.   7.   7.   6.   0.   9.]
 [ 17.  15.   5.   6.  16.  15.]
 [ 12.   0.   7.   7.  15.  11.]
 [  6.   4.   6.  13.  12.   5.]
 [  3.   8.  10.  16.  19.  16.]
 [  1.   3.   5.   1.  10.   4.]
 [ 19.   7.  10.  14.  18.  12.]
 [ 17.   7.   9.  16.  14.   5.]
 [  9.  18.  19.   3.  18.  12.]
 [  0.   7.  16.   1.  12.   2.]]
after 1st pivot
[[ 17.   7.   7.   6.   0.   9.]
 [ 17.  15.   5.   6.  16.  15.]
 [ 12.   0.   7.   7.  15.  11.]
 [  6.   4.   6.  13.  12.   5.]
 [  3.   8.  10.  16.  19.  16.]
 [  1.   3.   5.   1.  10.   4.]
 [ 19.   7.  10.  14.  18.  12.]
 [ 17.   7.   9.  16.  14.   5.]
 [  9.  18.  19.   3.  18.  12.]
 [  0.   7.  16.   1.  12.   2.]]
after 2nd pivot
[[  7.  17.   7.   6.   0.   9.]
 [  7.  17.   9.  16.  14.   5.]
 [  0.  12.   7.   

ValueError: Input matrix must not be singular

## Yet another bug

In [2]:
mat_test_6 = np.array([[ 3, 13,  4, 11,  6, 11],
       [14, 19, 16,  1,  4,  7],
       [17, 14, 18, 18,  9,  6],
       [ 7,  8,  8,  2, 16, 16],
       [10, 19,  3, 10, 18,  5],
       [ 1, 17,  9, 19, 15, 13],
       [13,  1,  4,  1, 17,  9],
       [10, 13,  8, 11,  3,  7],
       [ 4,  8,  6,  0,  1,  8],
       [ 8, 10, 10,  9,  1, 19]], dtype=float)

In [3]:
p,l,u,q = pluq_ids(mat_test_5, debug=True)

NameError: name 'mat_test_5' is not defined